In [1]:
from config import *
from iexfinance.stocks import get_historical_data
import requests
import datetime
import pandas as pd
import json

In [2]:
# Setup API to return assets
url = 'https://rest.coinapi.io/v1/assets'
headers = {'X-CoinAPI-Key' : API_KEY}
assets = requests.get(url, headers=headers)
assets = assets.json()

In [3]:
# Get list of altcoins from first 15 assets
Altcoins = []
for x in range(15):
    if assets[x]['type_is_crypto'] == 1:
        Altcoins.append(assets[x]['asset_id'])
print(Altcoins)

['BTC', 'ETH', 'LTC', 'USDT', 'XRP', 'BCH', 'ETC', 'XMR', 'NEO', 'EOS']


In [4]:
# Use Altcoins to get monthly data from coinAPI.io
altcoinData = []    
for coin in Altcoins:
    time_start = "2017-01-01"
    time_end ="2019-01-01"
    period_id = '1MTH'
    data_url = f'https://rest.coinapi.io/v1/ohlcv/{coin}/USD/history?period_id={period_id}&time_start={time_start}&time_end={time_end}'
    assets = requests.get(data_url, headers=headers)
    assets = assets.json()
    altcoinData.append(assets)

In [5]:
# Assign altcoinData to dataframe
altDF = pd.DataFrame(altcoinData)
altDF['altcoins'] = Altcoins
altDF.set_index('altcoins', drop=True, inplace=True)
altDF = altDF.transpose()
altDF.columns.names = ['']

In [6]:
# Drop altcoins that dont have data within specified timeframe
for coin in Altcoins:
    if altDF[coin][0]['time_period_start'] != '2017-01-01T00:00:00.0000000Z':
        altDF.drop(labels=coin, axis=1, inplace=True)

In [7]:
# Get daily data using only coins that have data within the specified timeframe
altcoins = altDF.columns.tolist()
altcoindata = []
for coin in altcoins:
    time_start = "2017-02-10" #time constraints determined by altcoin with least number of data points (XRP)
    time_end ="2019-02-10"
    period_id = '1DAY'
    limit = 1000
    data_url = f'https://rest.coinapi.io/v1/ohlcv/{coin}/USD/history?period_id={period_id}&time_start={time_start}&time_end={time_end}&limit={limit}'
    assets = requests.get(data_url, headers=headers)
    assets = assets.json()
    altcoindata.append(assets)

In [8]:
# Assign altcoins to their designated row and then transpose the rows and columns
altdf = pd.DataFrame(altcoindata)
altdf['altcoins'] = altcoins
altdf.set_index('altcoins', drop=True, inplace=True)
altdf = altdf.transpose()
altdf.columns.names = ['']

In [9]:
altdf.head()

,BTC,ETH,LTC,XRP,ETC,XMR
0,{'time_period_start': '2017-02-10T00:00:00.000...,{'time_period_start': '2017-02-10T00:00:00.000...,{'time_period_start': '2017-02-10T00:00:00.000...,{'time_period_start': '2017-02-10T00:00:00.000...,{'time_period_start': '2017-02-10T00:00:00.000...,{'time_period_start': '2017-02-10T00:00:00.000...
1,{'time_period_start': '2017-02-11T00:00:00.000...,{'time_period_start': '2017-02-11T00:00:00.000...,{'time_period_start': '2017-02-11T00:00:00.000...,{'time_period_start': '2017-02-11T00:00:00.000...,{'time_period_start': '2017-02-11T00:00:00.000...,{'time_period_start': '2017-02-11T00:00:00.000...
2,{'time_period_start': '2017-02-12T00:00:00.000...,{'time_period_start': '2017-02-12T00:00:00.000...,{'time_period_start': '2017-02-12T00:00:00.000...,{'time_period_start': '2017-02-12T00:00:00.000...,{'time_period_start': '2017-02-12T00:00:00.000...,{'time_period_start': '2017-02-12T00:00:00.000...
3,{'time_period_start': '2017-02-13T00:00:00.000...,{'time_period_start': '2017-02-13T00:00:00.000...,{'time_period_start': '2017-02-13T00:00:00.000...,{'time_period_start': '2017-02-13T00:00:00.000...,{'time_period_start': '2017-02-13T00:00:00.000...,{'time_period_start': '2017-02-13T00:00:00.000...
4,{'time_period_start': '2017-02-14T00:00:00.000...,{'time_period_start': '2017-02-14T00:00:00.000...,{'time_period_start': '2017-02-14T00:00:00.000...,{'time_period_start': '2017-02-14T00:00:00.000...,{'time_period_start': '2017-02-14T00:00:00.000...,{'time_period_start': '2017-02-14T00:00:00.000...


In [35]:
# Create dataframes for each individual altcoin
# Add new columns for machine learning algorithm to use in predicting closing prices
# Format Bitcoin dataframe
BTCList = altdf['BTC'].tolist()
BTC_df = pd.DataFrame(BTCList)
BTC_df['avg daily price'] = (BTC_df['price_high'] + BTC_df['price_low'])/2
BTC_df['avg daily price'] = BTC_df['avg daily price'].round(2)
BTC_df['usd traded'] = BTC_df['avg daily price'] * BTC_df['volume_traded']
BTC_df['% daily gain/loss'] = (BTC_df['price_close']-BTC_df['price_open'])/BTC_df['price_open'] *100

# Assign start time to index as "date"
dateList = BTC_df['time_period_start']
Start_Date = []
for date in range(len(dateList)):
    time = dateList[date].split('T')
    time = time[0].split('-')
    time = f'{time[1]}-{time[2]}-{time[0]}'
    Start_Date.append(time)
BTC_df['date'] = Start_Date
BTC_df['date'] =  pd.to_datetime(BTC_df['date'], format='%m-%d-%Y')
BTC_df.set_index('date', drop=True, inplace=True)

# Drop unwanted columns and rearrange remaining columns
BTC_df = BTC_df.drop(['time_close', 'time_open', 'time_period_end', 'time_period_start', 'trades_count'], axis=1)
BTC_df = BTC_df.rename(columns = {"price_close": "close","price_open":"open","price_high":"high","price_low":"low","volume_traded":"volume"})
BTC_df = BTC_df[['open','high','low','close','volume','% daily gain/loss','avg daily price','usd traded']]
BTC_df.to_csv('Data/BTC.csv')
BTC_df.head()

,open,high,low,close,volume,% daily gain/loss,avg daily price,usd traded
date,,,,,,,,
2017-02-10,987.431,1010.68,920.00,987.100,64033.664983,-0.033521,965.34,6.181426e+07
2017-02-11,995.490,1028.28,964.66,992.310,27810.748059,-0.319441,996.47,2.771258e+07
2017-02-12,992.310,1022.89,983.50,1000.730,13861.128046,0.848525,1003.20,1.390548e+07
2017-02-13,1001.590,1021.50,958.88,1000.360,28432.244497,-0.122805,990.19,2.815332e+07
2017-02-14,1000.740,1033.17,974.51,992.907,38062.882105,-0.782721,1003.84,3.820904e+07


In [36]:
# Format Ethereum dataframe
ETHList = altdf['ETH'].tolist()
ETH_df = pd.DataFrame(ETHList)
ETH_df['avg daily price'] = (ETH_df['price_high'] + ETH_df['price_low'])/2
ETH_df['avg daily price'] = ETH_df['avg daily price'].round(2)
ETH_df['usd traded'] = ETH_df['avg daily price'] * ETH_df['volume_traded']
ETH_df['% daily gain/loss'] = (ETH_df['price_close']-ETH_df['price_open'])/ETH_df['price_open'] *100


dateList = ETH_df['time_period_start']
Start_Date = []
for date in range(len(dateList)):
    time = dateList[date].split('T')
    time = time[0].split('-')
    time = f'{time[1]}-{time[2]}-{time[0]}'
    Start_Date.append(time)
ETH_df['date'] = Start_Date
ETH_df['date'] =  pd.to_datetime(ETH_df['date'], format='%m-%d-%Y')
ETH_df.set_index('date', drop=True, inplace=True)


ETH_df = ETH_df.drop(['time_close', 'time_open', 'time_period_end', 'time_period_start', 'trades_count'], axis=1)
ETH_df = ETH_df.rename(columns = {"price_close": "close","price_open":"open","price_high":"high","price_low":"low","volume_traded":"volume"})
ETH_df = ETH_df[['open','high','low','close','volume','% daily gain/loss','avg daily price','usd traded']]
ETH_df.to_csv('Data/ETH.csv')

In [37]:
# Format Litecoin dataframe
LTCList = altdf['LTC'].tolist()
LTC_df = pd.DataFrame(LTCList)
LTC_df['avg daily price'] = (LTC_df['price_high'] + LTC_df['price_low'])/2
LTC_df['avg daily price'] = LTC_df['avg daily price'].round(2)
LTC_df['usd traded'] = LTC_df['avg daily price'] * LTC_df['volume_traded']
LTC_df['% daily gain/loss'] = (LTC_df['price_close']-LTC_df['price_open'])/LTC_df['price_open'] *100


dateList = LTC_df['time_period_start']
Start_Date = []
for date in range(len(dateList)):
    time = dateList[date].split('T')
    time = time[0].split('-')
    time = f'{time[1]}-{time[2]}-{time[0]}'
    Start_Date.append(time)
LTC_df['date'] = Start_Date
LTC_df['date'] =  pd.to_datetime(LTC_df['date'], format='%m-%d-%Y')
LTC_df.set_index('date', drop=True, inplace=True)


LTC_df = LTC_df.drop(['time_close', 'time_open', 'time_period_end', 'time_period_start', 'trades_count'], axis=1)
LTC_df = LTC_df.rename(columns = {"price_close": "close","price_open":"open","price_high":"high","price_low":"low","volume_traded":"volume"})
LTC_df = LTC_df[['open','high','low','close','volume','% daily gain/loss','avg daily price','usd traded']]
LTC_df.to_csv('Data/LTC.csv')

In [38]:
# Format Ripple dataframe
XRPList = altdf['XRP'].tolist()
XRP_df = pd.DataFrame(XRPList)
XRP_df['avg daily price'] = (XRP_df['price_high'] + XRP_df['price_low'])/2
XRP_df['avg daily price'] = XRP_df['avg daily price'].round(2)
XRP_df['usd traded'] = XRP_df['avg daily price'] * XRP_df['volume_traded']
XRP_df['% daily gain/loss'] = (XRP_df['price_close']-XRP_df['price_open'])/XRP_df['price_open'] *100


dateList = XRP_df['time_period_start']
Start_Date = []
for date in range(len(dateList)):
    time = dateList[date].split('T')
    time = time[0].split('-')
    time = f'{time[1]}-{time[2]}-{time[0]}'
    Start_Date.append(time)
XRP_df['date'] = Start_Date
XRP_df['date'] =  pd.to_datetime(XRP_df['date'], format='%m-%d-%Y')
XRP_df.set_index('date', drop=True, inplace=True)


XRP_df = XRP_df.drop(['time_close', 'time_open', 'time_period_end', 'time_period_start', 'trades_count'], axis=1)
XRP_df = XRP_df.rename(columns = {"price_close": "close","price_open":"open","price_high":"high","price_low":"low","volume_traded":"volume"})
XRP_df = XRP_df[['open','high','low','close','volume','% daily gain/loss','avg daily price','usd traded']]
XRP_df.to_csv('Data/XRP.csv')

In [39]:
# Format Ethereum Classic dataframe
ETCList = altdf['ETC'].tolist()
ETC_df = pd.DataFrame(ETCList)
ETC_df['avg daily price'] = (ETC_df['price_high'] + ETC_df['price_low'])/2
ETC_df['avg daily price'] = ETC_df['avg daily price'].round(2)
ETC_df['usd traded'] = ETC_df['avg daily price'] * ETC_df['volume_traded']
ETC_df['% daily gain/loss'] = (ETC_df['price_close']-ETC_df['price_open'])/ETC_df['price_open'] *100


dateList = ETC_df['time_period_start']
Start_Date = []
for date in range(len(dateList)):
    time = dateList[date].split('T')
    time = time[0].split('-')
    time = f'{time[1]}-{time[2]}-{time[0]}'
    Start_Date.append(time)
ETC_df['date'] = Start_Date
ETC_df['date'] =  pd.to_datetime(ETC_df['date'], format='%m-%d-%Y')
ETC_df.set_index('date', drop=True, inplace=True)


ETC_df = ETC_df.drop(['time_close', 'time_open', 'time_period_end', 'time_period_start', 'trades_count'], axis=1)
ETC_df = ETC_df.rename(columns = {"price_close": "close","price_open":"open","price_high":"high","price_low":"low","volume_traded":"volume"})
ETC_df = ETC_df[['open','high','low','close','volume','% daily gain/loss','avg daily price','usd traded']]
ETC_df.to_csv('Data/ETC.csv')

In [40]:
# Format Monero dataframe
XMRList = altdf['XMR'].tolist()
XMR_df = pd.DataFrame(ETCList)
XMR_df['avg daily price'] = (XMR_df['price_high'] + XMR_df['price_low'])/2
XMR_df['avg daily price'] = XMR_df['avg daily price'].round(2)
XMR_df['usd traded'] = ETC_df['avg daily price'] * XMR_df['volume_traded']
XMR_df['% daily gain/loss'] = (XMR_df['price_close']-XMR_df['price_open'])/XMR_df['price_open'] *100


dateList = XMR_df['time_period_start']
Start_Date = []
for date in range(len(dateList)):
    time = dateList[date].split('T')
    time = time[0].split('-')
    time = f'{time[1]}-{time[2]}-{time[0]}'
    Start_Date.append(time)
XMR_df['date'] = Start_Date
XMR_df['date'] =  pd.to_datetime(XMR_df['date'], format='%m-%d-%Y')
XMR_df.set_index('date', drop=True, inplace=True)


XMR_df = XMR_df.drop(['time_close', 'time_open', 'time_period_end', 'time_period_start', 'trades_count'], axis=1)
XMR_df = XMR_df.rename(columns = {"price_close": "close","price_open":"open","price_high":"high","price_low":"low","volume_traded":"volume"})
XMR_df = XMR_df[['open','high','low','close','volume','% daily gain/loss','avg daily price','usd traded']]
XMR_df.to_csv('Data/XMR.csv')

In [16]:
# Get Stock Data
start = '2017-02-10'
end = '2019-02-10'

df_FB = get_historical_data("FB", start, end, output_format='pandas', token= api_key)
df_AMZN = get_historical_data("AMZN", start, end, output_format='pandas', token= api_key)
df_AAPL = get_historical_data("AAPL", start, end, output_format='pandas', token= api_key)
df_NFLX = get_historical_data("NFLX", start, end, output_format='pandas', token= api_key)
df_GOOGL = get_historical_data("GOOGL", start, end, output_format='pandas', token= api_key)
df_BIDU = get_historical_data("BIDU", start, end, output_format='pandas', token= api_key)
df_BABA = get_historical_data("BABA", start, end, output_format='pandas', token= api_key)
df_TCEHY = get_historical_data("TCEHY", start, end, output_format='pandas', token= api_key)

In [17]:
df_FB['%daily gain/loss'] = (df_FB['close']-df_FB['open'])/df_FB['open'] *100
df_AMZN['%daily gain/loss'] = (df_AMZN['close']-df_AMZN['open'])/df_AMZN['open'] *100
df_AAPL['%daily gain/loss'] = (df_AAPL['close']-df_AAPL['open'])/df_AAPL['open'] *100
df_NFLX['%daily gain/loss'] = (df_NFLX['close']-df_NFLX['open'])/df_NFLX['open'] *100
df_GOOGL['%daily gain/loss'] = (df_GOOGL['close']-df_GOOGL['open'])/df_GOOGL['open'] *100
df_BIDU['%daily gain/loss'] = (df_BIDU['close']-df_BIDU['open'])/df_BIDU['open'] *100
df_BABA['%daily gain/loss'] = (df_BABA['close']-df_BABA['open'])/df_BABA['open'] *100
df_TCEHY['%daily gain/loss'] = (df_TCEHY['close']-df_TCEHY['open'])/df_TCEHY['open'] *100

In [31]:
df_FB['avg daily price'] = (df_FB['high'] + df_FB['low'])/2
df_FB['avg daily price'] = df_FB['avg daily price'].round(2)
df_AMZN['avg daily price'] = (df_AMZN['high'] + df_AMZN['low'])/2
df_AMZN['avg daily price'] = df_AMZN['avg daily price'].round(2)
df_AAPL['avg daily price'] = (df_AAPL['high'] + df_AAPL['low'])/2
df_AAPL['avg daily price'] = df_AAPL['avg daily price'].round(2)
df_NFLX['avg daily price'] = (df_NFLX['high'] + df_NFLX['low'])/2
df_NFLX['avg daily price'] = df_NFLX['avg daily price'].round(2)
df_GOOGL['avg daily price'] = (df_GOOGL['high'] + df_GOOGL['low'])/2
df_GOOGL['avg daily price'] = df_GOOGL['avg daily price'].round(2)
df_BIDU['avg daily price'] = (df_BIDU['high'] + df_BIDU['low'])/2
df_BIDU['avg daily price'] = df_BIDU['avg daily price'].round(2)
df_BABA['avg daily price'] = (df_BABA['high'] + df_BABA['low'])/2
df_BABA['avg daily price'] = df_BABA['avg daily price'].round(2)
df_TCEHY['avg daily price'] = (df_TCEHY['high'] + df_TCEHY['low'])/2
df_TCEHY['avg daily price'] = df_TCEHY['avg daily price'].round(2)

In [29]:
df_FB['usd traded'] = df_FB['avg daily price'] * df_FB['volume']
df_AMZN['usd traded'] = df_AMZN['avg daily price'] * df_AMZN['volume']
df_AAPL['usd traded'] = df_AAPL['avg daily price'] * df_AAPL['volume']
df_NFLX['usd traded'] = df_NFLX['avg daily price'] * df_NFLX['volume']
df_GOOGL['usd traded'] = df_GOOGL['avg daily price'] * df_GOOGL['volume']
df_BIDU['usd traded'] = df_BIDU['avg daily price'] * df_BIDU['volume']
df_BABA['usd traded'] = df_BABA['avg daily price'] * df_BABA['volume']
df_TCEHY['usd traded'] = df_TCEHY['avg daily price'] * df_TCEHY['volume']

In [32]:
df_TCEHY.head()

,open,high,low,close,volume,%daily gain/loss,avg daily price,usd traded
date,,,,,,,,
2017-02-10,26.18,26.32,26.15,26.28,463200,0.381971,26.24,1.215205e+07
2017-02-13,26.30,26.39,26.25,26.33,491771,0.114068,26.32,1.294341e+07
2017-02-14,26.24,26.25,26.03,26.25,437777,0.038110,26.14,1.144349e+07
2017-02-15,26.55,26.78,26.37,26.73,988187,0.677966,26.58,2.626107e+07
2017-02-16,27.15,27.41,26.90,27.23,609656,0.294659,27.16,1.655521e+07


In [33]:
df_FB.index =  pd.to_datetime(df_FB.index, format='%Y-%m-%d')
df_AMZN.index =  pd.to_datetime(df_AMZN.index, format='%Y-%m-%d')
df_AAPL.index =  pd.to_datetime(df_AAPL.index, format='%Y-%m-%d')
df_NFLX.index =  pd.to_datetime(df_NFLX.index, format='%Y-%m-%d')
df_GOOGL.index =  pd.to_datetime(df_GOOGL.index, format='%Y-%m-%d')
df_BIDU.index =  pd.to_datetime(df_BIDU.index, format='%Y-%m-%d')
df_BABA.index =  pd.to_datetime(df_BABA.index, format='%Y-%m-%d')
df_TCEHY.index =  pd.to_datetime(df_TCEHY.index, format='%Y-%m-%d')

In [34]:
df_FB.to_csv('Data/FB.csv')
df_AMZN.to_csv('Data/AMZN.csv')
df_AAPL.to_csv('Data/AAPL.csv')
df_NFLX.to_csv('Data/NFLX.csv')
df_GOOGL.to_csv('Data/GOOGL.csv')
df_BIDU.to_csv('Data/BIDU.csv')
df_BABA.to_csv('Data/BABA.csv')
df_TCEHY.to_csv('Data/TCEHY.csv')